---

# CSCI 3202, Fall 2022
# Final Project
# Project Due: Thursday December 8, 2022 at 6:00 PM
## Proposals Due: Friday November 18, 2022 at 6:00 PM


You have two options for completing your final project for this course. 

#### Option 1 ####
The first option is presented in this notebook and involves implementing a Connect Four game with AB pruning and A* as player strategies. 

#### Option 2 ####
The second option is to design your own project that includes any of the algorithms we've discussed throughout the semester, or an algorithm that you're interested in learning that we haven't discussed in class. Your project also needs to include some kind of analysis of how it performed on a specific problem. If you're interested in the design your own project option, you need to discuss your idea with one of the course instructors to get approval. If you do a project without getting approval, you will receive a 0 regardless of the quality of the project. 

**The rules:**

1. Choose EITHER the given problem to submit OR choose your own project topic. 

2. If you choose your own project topic, please adhere to the following guidelines:
- Send an email to the course instructors before Friday, November 18 at 6pm, with a paragraph description of your project. We will respond within 24 hours with feedback.
- The project can include an algorithm we've discussed throughout the semester or an algorithm that you're been curious to learn. Please don't recycle a project that you did in another class. 
- If you do your own project without prior approval, you will receive a 0 for this project.
- Your project code, explanation, and results must all be contained in a Jupyter notebook. 

3. All work, code and analysis must be **your own**.
4. You may use your course notes, posted lecture slides, textbook, in-class notebooks and homework solutions as resources.  You may also search online for answers to general knowledge questions, like the form of a probability distribution function, or how to perform a particular operation in Python. You may not use entire segments of code as solutions to any part of this project, e.g. if you find a Python implementation of policy iteration online, you can't use it.
5. You may **not** post to message boards or other online resources asking for help.
6. **You may not collaborate with classmates or anyone else.**
7. This is meant to be like a coding portion of an exam. So, we will be much less helpful than we typically are with homework. For example, we will not check answers, help debug your code, and so on.
8. If you have a question, post it first as a **private** Piazza message. If we decide that it is appropriate for the entire class, then we will make it a public post (and anonymous).
9. If any part of the given project or your personal project is left open-ended, it is because we intend for you to code it up however you want. Our primary concern is with the plots/analysis that your code produces. Feel free to ask clarifying questions though.

Violation of these rules will result in an **F** and a trip to the Honor Code council.

---
**By writing your name below, you agree to abide by these rules:**

**Your name:**

---


---

Some useful packages and libraries:



In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time
from operator import itemgetter

# glopal vars 

num_states=0

---

## Problem 1: Game Theory - Playing "intelligent" Connect Four

Connect Four is a two-player game where the objective is to get four pieces in a row - horizontally, vertically, or diagonally. Check out this video if you're unfamiliar with the game: https://www.youtube.com/watch?v=utXzIFEVPjA.

### (1a)   Defining the Connect Four class structure

We've provided the humble beginnings of a Connect Four game. You need to fill in this class structure for Connect Four using what we did during class as a guide, and then implement min-max search with AB pruning, and A* search with at least one heuristic function. The class structure includes the following:

* `moves` is a list of columns to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`. Note that when a 'move' is made, the column must have an open slot and the piece must drop to the lowest row. 
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` returns the utility of the current state if the player is Red and $-1 \cdot$ utility if the player is Black.
* `display(self)` is a method to display the current game `state`. You get it for free because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if Red wins, 0 if draw, -1 if Black wins). `player1` and `player2` are functional arguments that we will deal with in parts **1b** and **1d**.

Some notes:
* Assume Red always goes first.
* Do **not** hard-code for 6x7 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [16]:
class State:
    def __init__(self, moves, r, c):
        self.to_move = 'R'
        self.utility = 0
        self.nrow = r
        self.ncol =c
        self.board = {}
        self.moves = moves
    def terminal_test(self):
        
        return len(self.moves)==0 or self.utility!=0

    def display(self):
        board = self.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()


        
class ConnectFour:
    def __init__(self, nrow=6, ncol=7, nwin=4):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = [(row,col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.state = State(moves, nrow, ncol)

    def result(self, move, state):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (R or B)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        if move not in self.actions(state): return state
        newmove =(0,0)
        new_state = cp.deepcopy(state)
        for i in reversed (range(1,self.nrow+1)):
            if (i, move[1]) not in new_state.board:
                new_state.board[(i, move[1])]= new_state.to_move
                newmove = (i, move[1])
                break
        
        if newmove==(0,0): raise KeyError
        new_state.utility= self.compute_utility(move, state)
        new_state.moves.remove(newmove)
        new_state.to_move= 'R' if state.to_move=='B' else 'B'

        return new_state

    def compute_utility(self, move, state):
        '''
        What is the utility of making move `move` in state `state`?
        If 'R' wins with this move, return 1;
        if 'B' wins return -1;
        else return 0.
        '''        
        
        # your code goes here...
        newmove=(0,0)
        # board = cp.deepcopy(state.board)
        new_state = cp.deepcopy(state)
        
        # new_state.display()
        for i in reversed (range(1,self.nrow+1)):
            if (i, move[1]) not in new_state.board:
                new_state.board[(i, move[1])]= new_state.to_move
                newmove =(i, move[1])
                break
        
        board = new_state.board
        row, col = newmove
        player = state.to_move
        ncol=self.ncol
        nrow=self.nrow
        nwin=self.nwin

             # check for row-wise win
        in_a_row = 0
        for c in range(1,ncol+1):
            x = board.get((row,c))==player
    
            if x is True:
                in_a_row +=1
                if in_a_row == nwin:
                    break
            else:
                in_a_row =0
             

        # check for column-wise win
        in_a_col = 0
        for r in range(1,nrow+1):
            if board.get((r,col))==player:
                in_a_col += 1
                if in_a_col ==nwin: break
            else:
                in_a_col=0

    #     # check for NW->SE diagonal win
        in_a_diag1 = 0
        for r in range(row,0,-1):
            if board.get((r,col-(row-r)))==player: 
                in_a_diag1 += 1
                if in_a_diag1 == nwin: break
            else:
                break

            
        for r in range(row+1,nrow+1):
            if board.get((r,col-(row-r)))==player: 
                in_a_diag1 += 1
                if in_a_diag1 == nwin: break
            else:
                break
        

    # # check for SW->NE diagonal win
        in_a_diag2 = 0
        for r in range(row,0,-1):
            if board.get((r,col+(row-r)))==player:
                in_a_diag2 += 1
                if in_a_diag2==nwin: break
            else:
                break
            
        for r in range(row+1,nrow+1):
            if board.get((r,col+(row-r)))==player:
                in_a_diag2 += 1
                if in_a_diag2==nwin: break
            else:
                break
        

        # print("new check ................. \n")
        # print(player)
        # print(newmove)    
        # print(nwin,in_a_row, in_a_col, in_a_diag1, in_a_diag2 )
        # new_state.display()
        if nwin in [in_a_row, in_a_col, in_a_diag1, in_a_diag2]:
            # print("winnnnnnnnnnnn")
            return 1 if player=='R' else -1
        else:
            return 0

    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''
        
        # your code goes here... 
        return( state.utility!=0 or len(state.moves)==0)  

    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''
        
        # your code goes here...
       
        if state.utility == 0:
            return 0  
        if state.utility == 1 and player == 'R':
            return 1   
        if state.utility == 1 and player == 'B':
            return -1
        if state.utility == -1 and player == 'R':
            return -1   
        if state.utility == -1 and player == 'B':
            return 1 

    def display(self):
        board = self.state.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()
    
    def actions(self, state):
        return state.moves
    def play_game(self, player1, player2):
        '''Play a game of Connect Four!'''
        
        # your code goes here...
        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0
        while turn<=turn_limit:
            for player in [player1, player2]:
                turn += 1
                move = player(self)
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
                    # self.display()
                    # print("last move", move)
                    return self.state.utility     

        
                

### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `ConnectFour` class and returns a random move out of the available legal moves in the `state` of the `ConnectFour` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [17]:
def random_player(game):
    '''A player that chooses a legal move at random out of all
    available legal moves in ConnectFour state argument'''
    
    # your code goes here...
    possible_actions = game.state.moves
    return possible_actions[np.random.randint(low=0, high=len(possible_actions))]
    


We know from experience and/or because I'm telling you right now that if two `random_player`s play many games of ConnectFour against one another, whoever goes first will win about 55% of the time.  Verify that this is the case by playing at least 1,000 games between two random players. Report the proportion of the games that the first player has won.**(Chris: is this true for TicTacToe, or Connect Four)**

**"Unit tests":** If you are wondering how close is close enough to 55%, I simulated 100 tournaments of 1,000 games each. The min-max range of win percentage by the first player was 52-59%.

In [18]:
# 1000 games between two random players

# Your code here

niter = 1000
wins = 0
draws = 0
losses = 0
for k in range(niter):
    game = ConnectFour()
    out = game.play_game(random_player, random_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
    # print("\n")
        
print('First-player winning percentage = {}'.format(wins/niter))

First-player winning percentage = 0.536


### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 10x10 ConnectFour tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **1b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

In [19]:
# 1000 games between two random players

# Your code here

niter = 1000
wins = 0
draws = 0
losses = 0
for k in range(niter):
    game = ConnectFour(10,10, 6)
    out = game.play_game(random_player, random_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
    
        
print('First-player winning percentage = {}'.format(wins/niter))

First-player winning percentage = 0.473


### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Connect Four game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `ConnectFour` class object and returns the minimax move in the `state` of the `ConnectFour` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [27]:


def alpha_beta_player(game):
    state = game.state
    return alpha_beta_search(state, game)

def minmax_player(game):
    state = game.state
    return minmax_decision(state, game)

def minmax_decision(state, game):
    """Given a state in a game, calculate the best move by searching
    forward all the way to the terminal states. [Figure 5.3]"""

    player = game.state.to_move
    global num_states
    # Functions used by alpha_beta
    def max_value(state: State):
        global num_states
        if game.game_over(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            num_states += 1
            v = max(v, min_value(game.result( a,state)))
            
        # state.display()
        return v

    def min_value(state: State):
        global num_states
        if game.game_over(state):
            # print("kgudjasf")
            return game.utility(state, player)
        v = np.inf
        # print("great1")
        for a in game.actions(state):
            num_states += 1
            # print(a)
            v = min(v, max_value(game.result(a, state)))
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    best_action = None
    
    for a in game.actions(state):
        # print("great0")
        num_states += 1
        v = min_value(game.result( a, state))
        # print("great")
        if v > best_score:
            best_score = v
            best_action = a
    return best_action
    

def alpha_beta_search(state, game):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = game.state.to_move
    global num_states
    # Functions used by alpha_beta
    def max_value(state: State, alpha, beta):
        global num_states
        if game.game_over(state):
            # print("kgud")
            # print(state.utility)
            # print(state.to_move)
            # state.display()
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            num_states += 1
            v = max(v, min_value(game.result( a,state), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        # game.display()
        return v

    def min_value(state: State, alpha, beta):
        global num_states
        if game.game_over(state):
            # print("kgudjasf")
            return game.utility(state, player)
        v = np.inf
        # print("great1")
        for a in game.actions(state):
            num_states += 1
            # print(a)
            v = min(v, max_value(game.result(a, state), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        # game.display()
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    
    for a in game.actions(state):
        num_states += 1
        # print("great0")
        v = min_value(game.result( a, state), best_score, beta)
        # print("great")
        if v > best_score:
            best_score = v
            best_action = a
    return best_action



Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 6x7 ConnectFour board. Run **10 games for each test**, and track the number of wins, draws and losses. Report these results for each case.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. Two alpha-beta players should always draw. One player is the max player and the other player is the min player.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. You are welcome to run more than 10 tests if you'd like. 

In [28]:
# Your code here
niter = 10
wins = 0
draws = 0
losses = 0
for k in range(niter):
    game = ConnectFour(3,3,3)
    out = game.play_game(alpha_beta_player, random_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
    # game.display()
    # print("\n")
        
print('First-player winning percentage = {} '.format(wins/niter), 'lossing ={}'.format(losses/niter))

First-player winning percentage = 0.8  lossing =0.0


In [29]:
# Your code here
niter = 10
wins = 0
draws = 0
losses = 0
for k in range(niter):
    game = ConnectFour(3,3,3)
    out = game.play_game(alpha_beta_player, alpha_beta_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
    # game.display()
    # print("\n")
        
print('Draws percentage = {} '.format(draws/niter))

Draws percentage = 1.0 


### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, **with and without pruning**, to determine the minimax decision from the initial 6x7 ConnectFour board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Then compute the percent savings that you get by using alpha-beta pruning. i.e. Compute $\frac{\text{Number of nodes expanded with pruning}}{\text{Number of nodes expanded with minimax}} $

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

In [30]:
# Your code here.
# Your code here
niter = 1
wins = 0
draws = 0
losses = 0
num_states=0
for k in range(niter):
    game = ConnectFour(3,3,3)
    out = game.play_game(minmax_player, random_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
    # game.display()
    # print("\n")
    print(num_states)
    num_states=0
        
print('player1 win percentage = {} '.format(wins/niter))

583025
player1 win percentage = 1.0 


In [31]:
# Your code here.
# Your code here
niter = 1
wins = 0
draws = 0
losses = 0
num_states=0
for k in range(niter):
    game = ConnectFour(3,3,3)
    out = game.play_game(alpha_beta_player, random_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
    print(num_states)
    num_states=0
    # game.display()
    # print("\n")
        
print('player1 win percentage = {} '.format(wins/niter))

8131
player1 win percentage = 1.0 


### (2) A* Search

In Part II of this project, you need to implement a player strategy to employ A* Search in order to win at ConnectFour. To test your A* player, play 10 games against the random player and 10 games against the AB pruning player. 

Write an explanation of this strategy's implementation and performance in comparison to the random player and the AB Pruning player from **1d**.

A lot of the code that you wrote for the minimax player and the Connect Four game structure can be reused for the A* player. However, you will need to write a new utility function for A* that considers the path cost and heuristic cost.


### (2a) Define a heuristic function
Your A* player will need to use a heuristic function. You have two options for heurstics: research an existing heuristic for Connect Four, or games similar to Connect Four, and use that. Or, design your own heuristic. Write a one-paragraph description of the heuristic you're using, including a citation if you used an existing heuristic.

### (2b) Compare A* to other algorithms
Next, play 10 games of Connect Four using your A* player and a random player and 10 games against the AB pruning player. In four or five paragraphs, report on the outcome. Did one player win more than the other? How often was the game a draw? How many moves did each player make? Were there situations where one player appeared to do better than the other? Given the outcome, are there other heuristics you would like to implement?

In [ ]:
# Your code here.